In [1]:
import sys
import os

# Add parent directory to sys.path
sys.path.append(os.path.abspath(".."))


In [ ]:
from models.spin_system import SpinSystem
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
import tensorflow as tf


In [ ]:

def run_temperature_sweep(N, J, betas, sweeps=200, theta_max=0.1):
    results = []

    for beta in betas:
        system = SpinSystem(N, J=J, keep_history=True)
        for _ in range(sweeps):
            system.metropolis_sweep(beta, steps=N, theta_max=theta_max)

        df = pd.DataFrame(system.history.get()).T
        df["beta"] = beta
        results.append(df)

    return pd.concat(results, ignore_index=True)

def plot_spins(sigmas, step=None):
    """
    Plot spins as a square heatmap.
    
    Parameters
    ----------
    sigmas : list of np.ndarray
        List of spin configurations (1D arrays).
    step : int or None
        Which step to plot (index in sigmas list). 
        If None, plot the last one.
    """
    if step is None:
        step = len(sigmas) - 1

    sigma = sigmas[step]
    N = len(sigma)

    # find closest square dimension
    side = int(math.ceil(math.sqrt(N)))
    padded = np.zeros(side**2)
    padded[:N] = sigma
    grid = padded.reshape(side, side)

    plt.figure(figsize=(6, 6))
    im = plt.imshow(grid, cmap="bwr", origin="upper")
    plt.colorbar(im, label="Spin value")
    plt.title(f"Spin configuration at step {step}")
    plt.axis("off")
    plt.show()

def make_spin_gif(sigmas, filename="spins.gif", interval=200):
    """
    Create a GIF showing the evolution of spins.

    Parameters
    ----------
    sigmas : list of np.ndarray
        List of spin configurations (1D arrays).
    filename : str
        Output filename for the gif.
    interval : int
        Delay between frames in milliseconds.
    """
    N = len(sigmas[0])
    side = int(math.ceil(math.sqrt(N)))

    # pad all sigmas to square
    grids = []
    for sigma in sigmas:
        padded = np.zeros(side**2)
        padded[:N] = sigma
        grids.append(padded.reshape(side, side))

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(grids[0], cmap="bwr", origin="upper", animated=True)
    plt.axis("off")

    def update(frame):
        im.set_array(grids[frame])
        ax.set_title(f"Step {frame}")
        return [im]

    ani = animation.FuncAnimation(fig, update, frames=len(grids),
                                  interval=interval, blit=True)

    ani.save(filename, writer="pillow")
    plt.close(fig)

In [ ]:
N = 50
J = tf.random.normal([N, N], stddev=1.0)
J = (J + tf.transpose(J)) / 2
J = tf.linalg.set_diag(J, tf.zeros(N))

betas = [0.1, 0.5, 1.0, 2.0, 5.0]

df = run_temperature_sweep(N, J, betas, sweeps=100)

plt.figure(figsize=(8,6))
for beta, group in df.groupby("beta"):
    group.reset_index().plot(
        y="magnetization", use_index=True, ax=plt.gca(), label=f"β={beta}"
    )

plt.xlabel("Step")
plt.ylabel("Magnetization ⟨m⟩")
plt.title("Magnetization evolution at different temperatures")
plt.legend()
plt.show()
